In [1]:
pip install -r ../../requirements.txt


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
%run -i ../../zkstats/core.py

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
# srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

## Step 1
Verifier calls `export_onnx` with dummy data, to generate the onnx model. Dummy data is used to infer the shape of the model

### Output
- `verifier.onnx` file

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
print("!@# len(data)=", len(data))
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer. Just need to be the same len as private data
dummy_data = np.round(np.random.uniform(1,10,len(data)),1)
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.mean(dummy_data_tensor)

!@# len(data)= 300


## Step 2
- User defines their computation in a function with signature `computation(state: State, x: list[torch.Tensor])`.
- Prover calls `create_model(computation)` to derive the actual model.
- Prover calls `prover_gen_settings`: export onnx file and compute the settings required by `ezkl.calibrate_settings`

In [6]:
from zkstats.computation import State, create_model


def computation(state: State, x: list[torch.Tensor]):
    x = x[0]
    b_0, out_0 = state.median(x)
    b_1, out_1 = state.median(x)
    b_2, out_2 = state.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))
    return torch.logical_and(torch.logical_and(b_0, b_1), b_2), out_2

prover_model = create_model(computation)
prover_gen_settings([data_path], comb_data_path, prover_model, prover_model_path, "default", "resources", settings_path)


/var/folders/t3/5psrvr1x0w1_6n9kx2n7d9700000gn/T/ipykernel_29423/1578668560.py:8: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  b_2, out_2 = state.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))
/var/folders/t3/5psrvr1x0w1_6n9kx2n7d9700000gn/T/ipykernel_29423/1578668560.py:8: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  b_2, out_2 = state.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))
/Users/mhchia/projects/work/pse/zk-stats-lib/.venv/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarni

==== Generate & Calibrate Setting ====
scale:  default
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":8,"param_scale":8,"scale_rebase_multiplier":10,"lookup_range":[-76800,153600],"logrows":18,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":11438,"total_const_size":1213,"model_instance_shapes":[[300,1],[1]],"model_output_scales":[0,8],"model_input_scales":[8],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}},"KroneckerDelta"],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


## Step 3
- Prover generates proof with `prover_gen_proof`:
    - 1. Compile circuit with `compile_circuit`
        - Inputs: onnx model, settings path
        - Outputs: compiled 
    - 2. Generate witness with
- Verifier generates keys with 

In [7]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(prover_model_path, prover_compiled_model_path, settings_path, vk_path, pk_path )

print("=======================================")
# Prover generates proof
# print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====


spawning module 0
spawning module 2
spawning module 0
spawning module 2


Time setup: 57.55826187133789 seconds
!@# compiled_model exists? True
!@# compiled_model exists? True
==== Generating Witness ====


spawning module 0


witness boolean:  1.0
witness result 1 : 49.55078125
==== Generating Proof ====


spawning module 2


proof:  {'instances': [[[3042937791208075219, 8157070662846698822, 3804781648660056856, 172406108020799675], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12436184717236109307

In [8]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path)

num_inputs:  1
prf instances:  [[[3042937791208075219, 8157070662846698822, 3804781648660056856, 172406108020799675], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12436184717